In [2]:
import numpy as np
import re
import pandas as pd
from pathlib import Path
import MeCab
import json
import requests


def clean_text(text):
    replaced_text = re.sub(r'[【】]', ' ', text)       # 【】の除去
    replaced_text = re.sub(r'[・_]', '', replaced_text)       # ・ の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    replaced_text = re.sub(r'\d+', '', replaced_text) # 数字の除去
    replaced_text = re.sub(r'[-/。,、.=]', ' ', replaced_text)
    return replaced_text

stop_words_ja = []
STOPWORD_FILE = Path("/home/iftekhar/amiebot/Helpbot/amie-HelpBot/amie_helpbot/assets/learning/stop_word.txt")
with open(STOPWORD_FILE, encoding='utf-8') as fr:
    stop_w = fr.read().splitlines()

newlist = ['ます',  'する', 'でき',  'です', 'ad', 'ください', 'この', 'あり',  'または', 'なり', 'その', 'ルール',]
stop_words_ja = stop_w + newlist

In [2]:
questions_word = []
questions_word_file = Path("/home/iftekhar/AI-system/JPBERT/questions_delimiter.txt")
with open(questions_word_file, encoding='utf-8') as f:
    questions_word_list = f.read().splitlines()


In [247]:
from bert_serving.client import BertClient
bc = BertClient()

### Just Example how does it do the encode

In [148]:
# bc.encode(['Hello World','How are you'])
bc.encode(['こんにちわ'])

array([[ 0.4426878 , -0.27872354, -2.2957592 , ...,  0.06125256,
        -0.03483357, -0.7257217 ]], dtype=float32)

In [3]:
df = pd.read_csv("~/AI-system/retrieval_Model/processed_perPage_perLine.csv")
df.head()

,Data,PageID
0,カスタマイザーからこのタイトルを設定できます。カスタマイザーからこのサブタイトルを設定できま...,0
1,Apple製品MobiControl v14 ManualApple製品をMobiContr...,1
2,Android端末MobiControl v14 ManualAndroidの設定モードA....,2
3,Windows PCMobiControl v14 ManualWindows PCの根本的...,3
4,Windows EmbeddedMobiControl v14 ManualWindows ...,4


In [4]:
# df = df.sample(frac=1).reset_index(drop=True)

In [5]:
df.head()

,Data,PageID
0,カスタマイザーからこのタイトルを設定できます。カスタマイザーからこのサブタイトルを設定できま...,0
1,Apple製品MobiControl v14 ManualApple製品をMobiContr...,1
2,Android端末MobiControl v14 ManualAndroidの設定モードA....,2
3,Windows PCMobiControl v14 ManualWindows PCの根本的...,3
4,Windows EmbeddedMobiControl v14 ManualWindows ...,4


In [6]:
#sample = df.head(20)
#sample

In [ ]:
df.iloc[[0]]
df.loc[0][0]

In [8]:
particles = ['る', '「 」', '「', '」', 'が', 'て', 'を', 'し', 'は', 
                   'か','に','の', 'へ', 'と', 'や', 'も', 'で'] 
df.Data = df.Data.apply(lambda x: clean_text(str(x)))

wakati = MeCab.Tagger("-Owakati")
df.Data = df.Data.apply(lambda x: wakati.parse(x).strip("\n"))
df.Data = df.Data.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words_ja]))
df.Data = df.Data.apply(lambda x: ' '.join(ch for ch in x.split() if ch not in particles))


In [10]:
df.head()
df.to_csv("/home/iftekhar/AI-system/retrieval_Model/processed_perPage_perLine.csv")

In [10]:
corpus_list = []
whole_corpus = []
n=100
word_limit = 10
count = 0
for index, row in df.iterrows():
    var = str(row['Data']).split('。')
    
    for elements in var:
        if len(elements) > n:
            elements = elements.split()
            # print(elements)
            # break
            chunks = [' '.join(elements[i:i+word_limit]) for i in range(0, len(elements), word_limit)]
            # print("splitted", chunks)
            # break
            for items in chunks:
                corpus_list.append([items,row['PageID']])            
        else:
            # print("not split",elements)
            corpus_list.append([elements,row['PageID']])
        # break
        
    # break    
    whole_corpus.append(corpus_list)
    corpus_list = []

In [11]:
whole_corpus[1]

[['Apple 製品 MobiControl v ManualApple 製品 MobiControl 管理 iOS 端末', 1],
 ['業務 アプリ 管理 MobiControl 業務 アプリ 管理 最適 ソリューション iOS', 1],
 ['端末 提供 端末 ユーザ 操作 なく アプリ サイレント インストール サイレント', 1],
 ['バージョンアップ 実現 端末 ユーザ 操作 待つ 必要 ない ので 短', 1],
 ['かい 時間 数 千 端末 アプリ 配布 バージョンアップ 業務 改善', 1],
 ['スピード 速め 社内 開発 App ストア 公開 たく ない アプリ', 1],
 ['端末 配布 インストールアプリ ホワイト リスト 作成 業務 関係 ない アプリ', 1],
 ['起動 禁止 社内 WiFi VPN リモート 設定 iOS 端末 画面', 1],
 ['リモートビューリモート iOS 端末 画面 コンソール 表示 端末 ユーザ アプリ 操作', 1],
 ['戸惑っ た アプリ 不具合 あっ た コンソール 画面 表示 対応', 1],
 ['v アドオン た アプリ リモートビュー 実現 ませ ん でし た', 1],
 ['v どの アプリ リモートビュー 可能 まし た macOS コンピュータ システム', 1],
 ['環境 設定 一括 設定 業務 アプリ サイレント インストール システム 環境', 1],
 ['設定 コンソール 一括 設定 ログイン 画面 設計 FileVault Dock Finder', 1],
 ['プリンタ Xsan 機能 拡張 面倒 な 設定 コンソール 一括 設定', 1],
 ['Active Directory Open Directory ネットワーク アカウント コンソール リモート 設定 ユーザ', 1],
 ['ホームフォルダ Time Machine による ファイル バックアップ サーバ リモート 設定 情報', 1],
 ['漏洩 対策 参照 例えば My Mac 禁止 職場 コンピュータ 自宅', 1],
 ['コンピュータ 情報 流出 防げ ネットワーク インフラ に対する アカウント 自動 設定', 1],
 ['社内 WiFi VPN 一斉 設定 

In [16]:
pages_list = []
content_list = []

count=0
for pages in whole_corpus:
    for sentences in pages:
        pages_list.append(sentences[0])
    content_list.append(pages_list)
    pages_list = []    
    

In [46]:
# This portion for splitting the corpus
split_content_corpus=[]
for pages in whole_corpus:
    for sentences in pages:
        pages_list.append([sentences[0],sentences[1]])
    split_content_corpus.append(pages_list)
    pages_list = []


len(split_content_corpus)
# for saving my split lines corpus
ids=[]
contents=[]
for i in range(len(split_content_corpus)):
    j=0
    for j in range(len(split_content_corpus[i])):
        ids.append(split_content_corpus[i][j][0])
        contents.append(split_content_corpus[i][j][1])

corpus_dataframe = pd.DataFrame(zip(contents, ids), columns=['IDs', 'Content'])
corpus_dataframe.to_csv("10words_line_corpus.csv")
corpus_dataframe.tail(100)

,IDs,Content
39089,443,端末 拾得 によって Exchange サーバ アクセス な 情報 漏洩 リスク
39090,443,無くす Exchange ActiveSync Filter インストール コンソール 該当...
39091,443,現われ た 画面 アイコ ン プルダウン 現われ Exchange アクセス 阻止
39092,443,クリック MobiControl サーバ Exchange ActiveSync Filte...
39093,443,送ら れ バー 全 OS タブ 選択 タブ サーバ タブ
...,...,...
39184,444,ページ 参照 オプション サーバ 機能 設定 端末 設定 端末 業務
39185,444,アプリ バージョンアップ VPP アカウント 設定 セキュリティ 対策 設定 分野 構成
39186,444,プロファイル 詳細 設定 端末 働きかける 端末 画面 メッセージ 表示 リモート
39187,444,ロック 紛失 モード リモート Wipe 初期 端末 働きかける 端末 状況


In [15]:
content_list[0]

['カスタマイザー タイトル 設定 カスタマイザー サブタイトル 設定 MobiControl 設定 順序 MobiControl',
 '設定 順序 MobiControl 設定 順序 行い 端末 OS 対象 オンプレミス',
 'MobiControl サーバ 必要 な 設定 事項 iOS 端末 macOS コンピュータ',
 '必須 接続 プロファイル 作成 ADDS Active Directory Domain Service サーバ',
 '間 ADFS Active Directory Federation Service 間 Azure AD 間',
 'Windows Windows Modern 管理 必須 会社 支給 端末 iOS Android',
 '複数 従業 共用 必須 Apple Google Microsoft 管理 アカウント 取得',
 '目的 Apple 社 サイト サインイン できる 権限 Apple 製品 設定',
 'App ストア アプリ 購入 managed Google Play アプリ アプリカタログ 使っ',
 '配布 つ アカウント 必要 但し Android 端末 Android Plus モード',
 '設定 アカウント 不要 WNS 使っ Windows PC リモート 管理 企業',
 '団体 組織 沿っ 階層 端末 グループ 作成 組織 入力 な',
 '作業 端末 OS 対象 端末 登録 作成 端末 所定 端末',
 'グループ 登録 端末 登録 目的 端末 ユーザ 認証 端末 型式',
 '設定 オプション ある つ 端末 グループ に対し 複数 端末 登録',
 '設定 可能 端末 OS 対象 端末 OS によって 作成 方法',
 '異なる 端末 挙動 に関する 設定 項目 設定 順序 変え 結構',
 '構成 プロファイル 作成 情報 漏洩 対策 WiFi VPN メール 設定',
 '端末 OS 対象 詳細 設定 設定 端末 OS 対象 端末',
 '登録 アプリカタログルール 作成 端末 OS 対象 アプリ 端末 配付 インストール',
 'Apple 製品 除く 端末 OS 対象 ドキュメント 画像 動画 端末',
 '配付 iOS 端末 Android 端末 対象 項 端末 挙動 

In [54]:
len(content_list[1])

219

In [55]:
    len(content_list[5])

20

In [56]:
len(content_list)

445

In [132]:
count = 0
all_question_list = []
for pages in content_list:
    question_list = []
    
    for sent in pages:
        sent = str(sent)
        
        if re.search(r'\w+', sent):
            #  print(count,"___",sent)
            item_list = []
            content = wakati.parse(sent).strip("\n").rstrip()
            item_list = content.split()
            item_list = list(set(item_list))
            #print(item_list)
            random_item_from_list = random.choice(item_list)
            item_list.remove(random_item_from_list)
            
            question = random.choice(questions_word_list)
            random.shuffle(item_list)
            item_list.append(question)
            question = ' '.join(item_list)
            question_list.append([question,count])
    count+=1
    all_question_list.append(question_list)
            
#         break   
#     break
        
        

In [133]:
len(all_question_list)

445

In [134]:
len(all_question_list[0])

127

In [135]:
all_question_list[0]

[['タイトル 順序 MobiControl 設定 サブタイトル 関する情報を教えてください', 0],
 ['設定 対象 順序 端末 オンプレミス 行い OS 入れられますか', 0],
 ['MobiControl 設定 コンピュータ iOS 端末 必要 事項 macOS サーバ どうして', 0],
 ['Domain 必須 Service 接続 サーバ 作成 Directory プロファイル ADDS どちら', 0],
 ['ADFS AD Service Federation Azure Directory 間 入れられますか', 0],
 ['Android iOS Modern 支給 Windows 端末 必須 管理 何ですか', 0],
 ['Microsoft 管理 Apple アカウント 取得 複数 Google 共用 必須 ことは何ですか', 0],
 ['できる イン 設定 社 Apple 目的 権限 サイト 製品 どの', 0],
 ['アプリカタログ 購入 使っ App アプリ Play Google ストア 何ですか', 0],
 ['つ 但し 端末 モード 配布 アカウント Android 必要 なにで', 0],
 ['WNS PC Windows 企業 不要 管理 アカウント 使っ リモート 使い始めるには', 0],
 ['沿っ 階層 グループ な 入力 端末 作成 組織 始める方法', 0],
 ['登録 所定 対象 作成 作業 OS 関する情報を教えてください', 0],
 ['認証 ユーザ 登録 型式 グループ 端末 ついて教えてください', 0],
 ['ある つ オプション グループ 登録 設定 に対し 端末 どう', 0],
 ['作成 対象 方法 設定 によって 可能 OS どこ', 0],
 ['結構 に関する 異なる 端末 順序 項目 変え 設定 どちら', 0],
 ['作成 メール 情報 設定 構成 対策 WiFi 漏洩 プロファイル 入れられますか', 0],
 ['対象 OS 端末 設定 どこですか', 0],
 ['アプリ 登録 対象 インストール 端末 作成 アプリカタログルール OS 始める方法', 0],
 ['OS 除く ドキュメント 端末 画像 Apple 対象 動画 どっち', 0],


In [140]:
labels = []
text_list = []
for i in all_question_list:
    for j in i:
        text_list.append(j[0])
        labels.append(j[1])
dataset = pd.DataFrame(zip(text_list, labels), columns=['Question','PageID'])
dataset.head()

,Question,PageID
0,タイトル 順序 MobiControl 設定 サブタイトル 関する情報を教えてください,0
1,設定 対象 順序 端末 オンプレミス 行い OS 入れられますか,0
2,MobiControl 設定 コンピュータ iOS 端末 必要 事項 macOS サーバ どうして,0
3,Domain 必須 Service 接続 サーバ 作成 Directory プロファイル A...,0
4,ADFS AD Service Federation Azure Directory 間 入...,0


In [ ]:
len(dataset)

In [204]:

def api_request(query):
    api_url = 'http://127.0.0.1:5000/api/v1.0/predict/'

    response = requests.post(api_url, 
                  data=[('output_dir', '/home/iftekhar/amiebot/Resources/876/model'),
                        ('data_dir', '/home/iftekhar/amiebot/Resources/876/data'),
                        ('agent_id', '-1'),
                        ('query', query),
                        ('max_answer', '3'),
                        ('user_nm', 'abc')
                       ]
                 )
    return response

#### Mean Reciprocal Rank is a measure to evaluate systems that return a ranked list of answers to queries. Actual_answer = [10] Predicted_answer=[20, 10, 100], Score=1/2,  if Predicted_answer=[10, 20, 100], score = 1, if Predicted_answer=[20, 100, 10], score = 1/3, and Predicted_answer=[20, 1020, 100], score=0, Then cumulative score is the total_accuracy https://en.wikipedia.org/wiki/Mean_reciprocal_rank


In [241]:
def Mean_reciprocal_rank_score(actual_answer, predicted_answers):
    pos = 0
    val = 0
    for i in predicted_answers:
        if i == actual_answer and pos==0:
            val = 1
            break
        elif i == actual_answer and pos==1:
            val = 0.5
            break
        elif i == actual_answer and pos==2:
            val = 0.33
            break
        else:
            val = 0
        pos+=1
    
    return val

In [243]:
import time 
c=0
sum_score=0
container=[]
for index, col in dataset.iterrows():
    response = api_request(str(col['Question']))
#     time.sleep(1)
    response.encoding='utf-8'
    res = response.json()
    y = json.loads(res)
    
    page_answers = [] 
    prediction_scores = []
    
#     print("Line: ", c)
    
    for i in y['page_relevance']:
        parsed_text = ' '.join(i['keywords'])       
        page_answers.append(i['doc_id'])
        prediction_scores.append(i['score'])
    
    MRR_score = Mean_reciprocal_rank_score(col['PageID'], page_answers)
    sum_score+=MRR_score
    container.append([MRR_score, col['PageID'], page_answers, prediction_scores, parsed_text, col['Question']])
#     print(MRR_score,",", col['PageID'], ",",page_answers,",",prediction_scores,",", parsed_text, ",", col['Question'])
    
#     if c>10:
#         break
    c+=1
Total_accuracy = sum_score/c  

KeyboardInterrupt: 

In [250]:
    score_list=[]
    actual_answer_list = []
    page_answers_list = []
    prediction_scores = []
    prediction_text = []
    query_list = []

    for items in container:
        score_list.append(items[0])
        actual_answer_list.append(items[1])
        page_answers_list.append(items[2])
        prediction_scores.append(items[3])
        prediction_text.append(items[4])
        query_list.append(items[5])

    d = pd.DataFrame()
    d['score'] = score_list 
    d['actual_answer'] = actual_answer_list
    d['page_answers'] = page_answers_list
    d['prediction_scores'] = prediction_scores
    d['prediction_text'] = prediction_text
    d['query_list'] = query_list


In [252]:
d.head()
    d.to_csv('performance.csv')

In [248]:
container[0]

[1,
 0,
 [0, 188, 112],
 [21.047790171749877, 14.66680545792326, 14.636287632112722],
 'タイトル 順序 mobicontrol 設定 情報',
 'タイトル 順序 MobiControl 設定 サブタイトル 関する情報を教えてください']

In [213]:
page_answers

[15, 362, 223]

# I will work later from below


In [ ]:
count=0
encoded_vectors = []
pages_vector = []
for page_list in content_list:
    sentences_vector = bc.encode(page_list)
    pages_vector.append(sentences_vector)                   

In [2]:
import pickle

# Not writing on disk because already it is present there
# with open('Bert_vec_per_page.pkl', 'wb') as f:
#     pickle.dump(pages_vector ,f)
    
with open('Bert_vec_per_page.pkl', 'rb') as f:
    vector = pickle.load(f)
    
len(vector)    

445

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

# Read a question from the user
ori_question = [input('Please enter a question: \n')]
question = bc.encode(ori_question)

Please enter a question: 
下記の項目の設定順序を変えても結構です


In [115]:
question
type(question)
question.ndim

2

In [7]:
sent_score_list = []
page_score_list = []
for i in vector:
    for j in i:
        j=j.reshape(1, -1)
        sentence_score = cosine_similarity(question, j)
        sent_score_list.append(sentence_score)
    page_score_list.append(sent_score_list)
    sent_score_list = []

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 1024 while Y.shape[1] == 768

In [1]:
len(page_score_list[0])

NameError: name 'page_score_list' is not defined

In [125]:
print(question.shape)
j=j.reshape(1,-1)
j.shape

(1, 768)


(1, 768)

### This part was recommended in the BERT service github


##### Running the server  bert-serving-start -model_dir /tmp/english_L-12_H-768_A-12/ -num_worker=4 
https://github.com/hanxiao/bert-as-service

In [ ]:
# Rank all the questions using dot product to the input question
while True:
    query = input('your question: ')
    query_vec = bc.encode([query])[0]
    # compute normalized dot product as score
    score = np.sum(query_vec * doc_vecs, axis=1) / np.linalg.norm(doc_vecs, axis=1)
    topk_idx = np.argsort(score)[::-1][:topk]
    for idx in topk_idx:
        print('> %s\t%s' % (score[idx], questions[idx]))


In [13]:



# Grab the top 5
top = np.argsort(rank, axis=-1).T[-10:]
#print(top)
また、端末OSによっては、MobiControlエージェントソフトをダウンロードし、これに「登録ID」を 入力することでも、端末は所定の端末グループに登録されます
for item in range(len(top)):
    idx = top[item,0]
    print(idx)
    
#     # if the dataset file structure "Quesition | Answer" then use following line  
#     #print(data['Answer'].iloc[item].values[0])
#     print("PageID: ",result.iloc[idx,-1],"idx",idx,"=",tog.values[idx])
#     print("\n ########## \n")

3
2
0
1


In [14]:
top

array([[3],
       [2],
       [0],
       [1]])

In [15]:
rank

array([[0.92932934, 0.93377185, 0.8743946 , 0.86578614]], dtype=float32)

In [138]:
print('Question:', ori_question)
print('Answers:')
count=0
score = []
text = []
for i in rank:
    for j in i:
        print(j,"-",input_array[count])
        score.append(j)
        text.append(input_array[count])
        count+=1

Question: ['また、端末OSによっては、MobiControlエージェントソフトをダウンロードし、これに「登録ID」を 入力することでも、端末は所定の端末グループに登録されます']
0.92932934 - カスタマイザーからこのタイトルを設定できます
0.93377185 - カスタマイザーからこのサブタイトルを設定できます
0.8743946 - MobiControlの設定順序MobiControlの設定順序MobiControlの設定は次の順序で行います
0.86578614 - 端末の挙動に関する設定


In [33]:
import pandas as pd
df = pd.DataFrame(zip(score,text), columns=['a','b'])
df.head()

,a,b
0,0.929329,カスタマイザーからこのタイトルを設定できます
1,0.933772,カスタマイザーからこのサブタイトルを設定できます
2,0.874395,MobiControlの設定順序MobiControlの設定順序MobiControlの設定...
3,0.865786,端末の挙動に関する設定


In [35]:
df.sort_values(by=['a'], ascending=False)

,a,b
1,0.933772,カスタマイザーからこのサブタイトルを設定できます
0,0.929329,カスタマイザーからこのタイトルを設定できます
2,0.874395,MobiControlの設定順序MobiControlの設定順序MobiControlの設定...
3,0.865786,端末の挙動に関する設定
